In [1]:

import gym
#from gym.spaces import Discrete, MultiDiscrete
import numpy as np
import random

from ray.rllib.env.multi_agent_env import MultiAgentEnv


class MultiMinerMarket(MultiAgentEnv):
    def __init__(self , config = None):
        config = config or {}
        # define reward
        self.R = 100
        self.marginal_cost = [1,1,1,1,1]
        #number of agents
        self.num_agents = len(self.marginal_cost)
        # define a list for hash values
        self.hashes = [0]*self.num_agents
        # define list of probabilities
        self.probs = [0] * self.num_agents
        # observation_space: previous probability , previous hash
        self.observation_space = gym.spaces.Dict({
            "agent_" + str(i) : gym.spaces.Box(low=np.array([0 , 0]), high=np.array([1 , self.R/self.marginal_cost[i]]), dtype=np.float32)  for i in range(0,self.num_agents)
        })
        
        
        self.action_space = gym.spaces.Dict({
            "agent_" + str(i) : gym.spaces.Box(low=np.array([0]), high=np.array([self.R/self.marginal_cost[i]]), dtype=np.float32)  for i in range(0,self.num_agents)
        })
        
        self.reset()
        
    def reset(self):
        #obser = self._get_obs()
        
        #self.marginal_valuation_vector = np.random.uniform(0,10,self.num_agents).tolist()
        
        #self.action_space = gym.spaces.Dict({
         #   "agent_" + str(i) : gym.spaces.Box(low=np.array([0]), high=np.array([self.marginal_valuation_vector[i]]), dtype=np.float32)  for i in range(0,self.num_agents)
        #})
        
        return self._get_obs()
    

    def step(self, action: dict):
        
        # calculate hashes
        hashes = []
        for i in range(0 , self.num_agents):
            hashes.append(action["agent_" + str(i)][0])
            
        self.hashes = hashes
            
        # calculate prs as function of hashes
        if max(hashes)==0:
            prs = [0]*self.num_agents # exclude cases where all the chosen actions are zeros.
        else:
            prs = [hashes[k]/sum(hashes) for k in range(self.num_agents)] # 0<prs[i]<1  
            
        self.probs = prs
            
        # calculate rewards    
        rew = [prs[l] * self.R - self.marginal_cost[l] * hashes[l] for l in range(self.num_agents)]
            
         
        
        rewards = {
             "agent_" + str(i) : float(rew[i]) for i in range(0,self.num_agents)
        }

        obs = self._get_obs()
        
        
        is_done = True
        
        dones = {
           "agent_0" : is_done,
            "agent_1" : is_done,
            "agent_2" : is_done,
            "agent_3" : is_done,
            "agent_4" : is_done,
            # special `__all__` key indicates that the episode is done for all agents.
            "__all__": is_done,
        }
        
        return obs, rewards, dones, {}  # <- info dict (not needed here).

    def _get_obs(self):
        return {
            "agent_" + str(i) : [self.probs[i] ,  self.hashes[i]] for i in range(0,self.num_agents)
        }

    def render(self, mode=None):
        pass


In [2]:
game = MultiMinerMarket()

C:\Users\mohazaa1\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [3]:
#test
game = MultiMinerMarket()
game.reset()
count = 0
while count < 1:
    count = count + 1
    action = game.action_space.sample()
    hashes = []
    for i in range(0 , game.num_agents):
        hashes.append(action["agent_" + str(i)][0])
    #print("marginal_value" , game.marginal_valuation_vector)
    #print("bids" , bids)
    obs , rew , done , info = game.step(action)
    for i in range(0 , game.num_agents):
        print(hashes[i] , game.probs[i] , rew["agent_" + str(i)])
    game.reset()

58.9723 0.2877267818239358 -30.199623300760717
92.12544 0.44948147569936353 -47.17729493494646
35.14807 0.17148798986943156 -17.999272302119344
12.238783 0.059713213215602585 -6.267461561130171
6.474777 0.03159053939166653 -3.315723282513034


In [43]:
game.reset()
a = game.observation_space
print(a['agent_0'])

TypeError: 'Box' object is not subscriptable

In [39]:
a['agent_0'][0]

81.48279

In [4]:
import numpy as np
import pprint
import ray

# Start a new instance of Ray (when running this tutorial locally) or
# connect to an already running one (when running this tutorial through Anyscale).

ray.init()  # Hear the engine humming? ;)


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:57849',
 'raylet_socket_name': 'tcp://127.0.0.1:62851',
 'webui_url': None,
 'session_dir': 'C:\\Users\\mohazaa1\\AppData\\Local\\Temp\\ray\\session_2022-10-25_10-24-21_504707_13524',
 'metrics_export_port': 58690,
 'node_id': '18fec58ab09ac3434c4883db42da334d010b457f4d1cfbc3f0127551'}

In [5]:
from ray.rllib.agents.ppo import PPOTrainer


policies= {
    "policy_" + str(i): (None , game.observation_space["agent_" + str(i)] , game.action_space["agent_" + str(i)] , {"gamma" : 0.04}) for i in range(0,game.num_agents)
}

def policy_mapping_fn (agent_id: str):
    #assert agent_id in [str(i) for i in range(0,5)], f"ERROR: invalid agent id {agent_id}!!!"
    return "policy_" + str(agent_id[len(agent_id)-1])
        
config={
    "env": MultiMinerMarket,  # "my_env" <- if we previously have registered the env with `tune.register_env("[name]", lambda config: [returns env object])`.
    #"framework": "torch",
    "model":{
        "fcnet_hiddens": [512 , 512],
    },
    "num_workers": 4,
    "create_env_on_driver": True,
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": policy_mapping_fn,
},
}




In [54]:
# Override the DefaultCallbacks with your own and implement any methods (hooks)
# that you need.
from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.evaluation.episode import MultiAgentEpisode


class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self,
                         *,
                         worker,
                         base_env,
                         policies,
                         episode: MultiAgentEpisode,
                         env_index,
                         **kwargs):
        # We will use the `MultiAgentEpisode` object being passed into
        # all episode-related callbacks. It comes with a user_data property (dict),
        # which we can write arbitrary data into.

        # At the end of an episode, we'll transfer that data into the `hist_data`, and `custom_metrics`
        # properties to make sure our custom data is displayed in TensorBoard.

        # The episode is starting:
        # Set per-episode object to capture, which states (observations)
        # have been visited by agent1.
        #episode.user_data["market_clearing_price"] = 0
        # Set per-episode agent2-blocks counter (how many times has agent2 blocked agent1?).
        #episode.user_data["average_accepted_bid"] = 0
        pass
        

    def on_episode_step(self,
                        *,
                        worker,
                        base_env,
                        episode: MultiAgentEpisode,
                        env_index,
                        **kwargs):

        pass

    def on_episode_end(self,
                       *,
                       worker,
                       base_env,
                       policies,
                       episode: MultiAgentEpisode,
                       env_index,
                       **kwargs):
        # Episode is done:
        # Write scalar values (sum over rewards) to `custom_metrics` and
        # time-series data (rewards per time step) to `hist_data`.
        # Both will be visible then in TensorBoard.
        
        #res = []
        #for i in range(0 , game.num_agents):
         #   if episode.last_observation_for("agent_" + str(i))[1]!=0:
          #      res.append(episode.last_observation_for("agent_" + str(i))[0] - episode.last_observation_for("agent_" + str(i))[1])  
        episode.user_data["agent_0_hash"] = episode.last_observation_for('agent_0')[1]
        episode.user_data["agent_1_hash"] = episode.last_observation_for('agent_1')[1]
        episode.user_data["agent_2_hash"] = episode.last_observation_for('agent_2')[1]
        episode.user_data["agent_3_hash"] = episode.last_observation_for('agent_3')[1]
        episode.user_data["agent_4_hash"] = episode.last_observation_for('agent_4')[1]
        
        episode.custom_metrics["agent_0_hash"] = episode.user_data["agent_0_hash"]
        episode.custom_metrics["agent_1_hash"] = episode.user_data["agent_1_hash"]
        episode.custom_metrics["agent_2_hash"] = episode.user_data["agent_2_hash"]
        episode.custom_metrics["agent_3_hash"] = episode.user_data["agent_3_hash"]
        episode.custom_metrics["agent_4_hash"] = episode.user_data["agent_4_hash"]
        

In [55]:
from ray import tune

tune_config = config.copy()
tune_config["callbacks"] = MyCallbacks

tune.run(
    "PPO",
    config=tune_config,
    stop={"training_iteration": 100},
    checkpoint_at_end=True,
    checkpoint_freq=5,
    # If you'd like to restore the tune run from an existing checkpoint file, you can do the following:
    #restore="/Users/sven/ray_results/PPO/PPO_MultiAgentArena_fd451_00000_0_2021-05-25_15-13-26/checkpoint_000010/checkpoint-10",
    local_dir = "5_miners_12"
)

Trial name,status,loc
PPO_MultiMinerMarket_056e3_00000,PENDING,


 pid=8196) 2022-10-25 14:42:12,077	INFO trainer.py:722 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
 pid=8196) 2022-10-25 14:42:12,077	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
 pid=8196) 2022-10-25 14:42:12,078	INFO trainer.py:743 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=8196) C:\Users\mohazaa1\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
 pid=8196)   warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
 pid=8196) 2022-10-25 14:42:12,527	WARNING deprecation.p

Trial name,status,loc
PPO_MultiMinerMarket_056e3_00000,RUNNING,127.0.0.1:8196


 pid=8196) 2022-10-25 14:42:17,952	WARNING util.py:57 -- Install gputil for GPU system monitoring.
 pid=14760) 2022-10-25 14:42:18,162	WARNING deprecation.py:45 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` instead. This will raise an error in the future!


Trial name,status,loc
PPO_MultiMinerMarket_056e3_00000,RUNNING,127.0.0.1:8196


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiMinerMarket_056e3_00000,RUNNING,127.0.0.1:8196,53,2315,212000,19.7144,30.6843,9.06822,1


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiMinerMarket_056e3_00000,RUNNING,127.0.0.1:8196,53,2315,212000,19.7144,30.6843,9.06822,1


2022-10-25 15:21:37,830	WARNING tune.py:582 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiMinerMarket_056e3_00000,RUNNING,127.0.0.1:8196,53,2315,212000,19.7144,30.6843,9.06822,1


2022-10-25 15:21:38,246	ERROR tune.py:622 -- Trials did not complete: [PPO_MultiMinerMarket_056e3_00000]
2022-10-25 15:21:38,247	INFO tune.py:626 -- Total run time: 2384.92 seconds (2384.50 seconds for the tuning loop).
2022-10-25 15:21:38,248	WARNING tune.py:630 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
